In [1]:
# Libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Read in all DFs
dat = pd.read_csv("./data/processed_data.csv")
vax = pd.read_csv("./data/processed_vax.csv")
sym = pd.read_csv("./data/processed_symptoms.csv")

In [3]:
# Pull only information from vax DF which could be sensibly used given we need to get rid of duplicates
vax = pd.DataFrame(vax.groupby("vaersId")["covid"].apply(sum))

# We make a single column vax DF which represents number of Covid vaccines received
vax = vax.reset_index()
vax.columns = ["vaersId", "nCovidVax"]

In [4]:
# Merge all three dataframes together, combining X (features) and y (target)
sigma = dat.merge(vax, on="vaersId").merge(sym, on="vaersId")

In [8]:
sigma

,vaersId,age,sex,recovered,deltaOnset,adminBy,otherMeds,history,region,deltaReceived,nCovidVax,s1,s2,s3,s4,s5,myocarditis
0,916600,33.0,F,Y,2.0,1,0,0,1,2.0,1,4839,5510.0,0.0,0.0,0.0,0
1,916601,73.0,F,Y,0.0,7,1,1,2,1.0,1,1161,7639.0,0.0,0.0,0.0,0
2,916604,47.0,F,N,7.0,4,0,0,1,3.0,1,2584,6797.0,6347.0,1940.0,0.0,0
3,916606,44.0,F,Y,0.0,1,0,0,2,3.0,1,4173,0.0,0.0,0.0,0.0,0
4,916607,50.0,M,Y,1.0,4,1,2,3,3.0,1,33,5567.0,714.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279839,1996866,31.0,F,N,1.0,1,1,0,3,1.0,0,6645,0.0,0.0,0.0,0.0,0
279840,1996867,83.0,F,N,2.0,7,1,1,4,300.0,1,4548,1192.0,8438.0,6746.0,7946.0,0
279841,1996868,16.0,M,Y,1.0,4,0,0,3,1.0,1,8430,7364.0,8504.0,2063.0,0.0,0
279842,1997117,18.0,F,N,3.0,2,0,1,4,4.0,1,6645,0.0,0.0,0.0,0.0,0


In [5]:
# Write df to csv
sigma.to_csv("./data/combined.csv", index=False)